In [14]:
import tensorflow as tf
tf2x=tf.compat.v1
import numpy as np
from scipy.special import gamma
"""
For consitancy of operations in tendorflow:
1. Make sure everything - evenry input is in float32 and all operations are on float32
"""

'\nFor consitancy of operations in tendorflow:\n1. Make sure everything - evenry input is in float32 and all operations are on float32\n'

In [15]:
def pairwise_dist (A, B):  
    
    """
  Computes pairwise distances between each elements of A and each elements of B.
  Args:
    A:    [m,d] matrix
    B:    [n,d] matrix
  Returns:
    D:    [m,n] matrix of pairwise distances
           If A is a single element then we get an array of vectors  
    """
    with tf2x.variable_scope('pairwise_dist'):
    # squared norms of each row in A and B
        na = tf2x.reduce_sum(tf.square(A), 1)
        nb = tf2x.reduce_sum(tf.square(B), 1)
    
    # na as a row and nb as a column vectors
    na = tf2x.reshape(na, [-1, 1])
    nb = tf2x.reshape(nb, [1, -1])

    # return pairwise euclidean diance matrix or a vector of distances
    D = tf.sqrt(tf.maximum(na - 2*tf.matmul(A, B, False, True) + nb, 0))
    return D

In [47]:
def trans_func(i):
    global d,m
    if d%2==0:
        nu_md_num=(i**((2*m)-d))*((-1)**(m+1+(d/2)))*(np.log(i))
        nu_md_deno=(2**(2*m-1))*(np.pi**(d/2))*(np.math.factorial(m-1))*(np.math.factorial(m-(d/2)))
        k=nu_md_num/nu_md_deno
        #trans.append(k)
    else:
        
        nu_md_num=(i**((2*m)-d))*(gamma((d/2)-m))
        nu_md_deno=(2**(2*m))*(np.pi**(d/2))*(np.math.factorial(m-1))
        k=nu_md_num/nu_md_deno
        
        
    return k

    

In [48]:
m=1
d=2
elems = np.array([1, 2., 3, 4, 5, 6])
a=tf.map_fn(trans_func,elems)
print(a)

tf.Tensor([-0.         -0.1103178  -0.17484958 -0.2206356  -0.25615    -0.28516738], shape=(6,), dtype=float64)


In [49]:
def tps(r_ast,c_ast,r_nov,c_nov,offset,Xu,beta=[3,4],phi=[[1,0,0],[0,1,0],[0,0,1]],m=3,d=2,uz_in=[1.,1,1,1,1,1,1,1],Q_qr=[1,.1,1,1]):
    """
    offset: It is an array of 2 dimension that basically denotes the alpha
    Xu: An array of 2d points  
    phi: The thin plate spline matrix
         Note its form: It is of form list of lists
    
    """
    x_nov=tf.constant([r_nov,c_nov])
    x_ast=tf.constant([r_ast,c_ast])
    
    ## This is the first step from the TPS optimization
    x_cap_nov=x_ast-x_nov
    
    ## Applying the offset this is the second step
    alpha=tf.constant(offset,dtype='float32')
    x_til_nov=x_cap_nov-alpha
    x_til_nov=tf.cast(x_til_nov,tf.float32)
    
    ##L-2 norm step
    ## D is the vector of eucledian distances
    c= x_til_nov.numpy()
    lst = [[None]]
    data = np.array(Xu, dtype=np.float32)
    lst[0]=c
    D=pairwise_dist (lst, data)
    
    ##Transform the matrix
    """
    The vector trans stores the transformed eucledean distance
    """
    arr=D.numpy()
    trans_tesnor=tf.map_fn(trans_func,elems)
#     trans=[]
#     if d%2==0:
#         for i in arr:
#             nu_md_num=(i**((2*m)-d))*((-1)**(m+1+(d/2)))*(np.log(i))
#             nu_md_deno=(2**(2*m-1))*(np.pi**(d/2))*(np.math.factorial(m-1))*(np.math.factorial(m-(d/2)))
#             k=nu_md_num/nu_md_deno
#             trans.append(k)
#     else:
#         for i in arr:
#             nu_md_num=(i**((2*m)-d))*(gamma((d/2)-m))
#             nu_md_deno=(2**(2*m))*(np.pi**(d/2))*(np.math.factorial(m-1))
#             k=nu_md_num/nu_md_deno
#             trans.append(k)
    
    ## The polynomial computation
    trans=trans_tesnor.numpy()
    
    """
    T is the computed polynomial
    """
    T=[]
    c=x_nov.numpy()
    for i in range(m):
        ti=1
        for j in range(len(c)):
            ti=ti*(c[j]**phi[i][j])
        T.append(ti)
    return trans
    
    ###The appending step
    """
    Note Here: E is a list of numbers
    Zp is the actual vector that we need to pass to the spectral deomposition step
    
    """
    map(float,T)
    E=[]
    E=trans+T
    arr=np.array(E)
    c=np.array(arr)
    zp=[]
  
    for i in c[0]:
        zp.append(i)
    for i in range((len(c)-1)):
        zp.append(c[i+1])
    
    
    
    ###Reconstructing Z from Spectral decompostion
    """
    Here the Inputs are matrix Q and matrix UZ
    """
    E_vec=tf.constant(zp, shape=[4])
    UZ=tf.constant(uz_in, shape=[2,4])
    Q=tf.constant(Q_qr,shape=[2,2])
    K=tf.linalg.matvec(UZ,E_vec)
    Z=tf.linalg.matvec(Q,K)
    p=Z.numpy()
    ## Update first variable of Z per intercept term:
    p[0]=1
    
    
    ## Getting the intercept
    exponent=np.dot(beta,p)
    y_hat=np.exp(exponent)
    return y_hat
    
 
            
            
    
    

In [50]:

tps(2.3,3.7,5.6,6,[8,7],[[2,3]])

array([-0.        , -0.1103178 , -0.17484958, -0.2206356 , -0.25615   ,
       -0.28516738])

In [497]:
x=tf.constant([1,3])
y=tf.constant([[1,3],[5,6]])
t=x-y
r = tf.reduce_sum(y*y, 1)
r = tf.reshape(r, [-1, 1])


In [182]:
t*t

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[ 0,  0],
       [16,  9]], dtype=int32)>

In [195]:
np.math.factorial(5)


120

In [183]:
D

<tf.Tensor: shape=(1, 2), dtype=int32, numpy=array([[ 4, 13]], dtype=int32)>

In [221]:

pairwise_dist([[1.0,3.0]],[[1.0,3.0],[5.0,6.0]])

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 5.]], dtype=float32)>

In [239]:

z=[[1,2,3],[4,5,6],[7,8,9]]
for i in range(3):
    for j in range(3):
        print(z[i][j])

1
2
3
4
5
6
7
8
9


In [417]:
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])

# 1-D tensor `b`
# [7, 9, 11]
b = tf.constant([7, 9, 11], shape=[3])

# `a` * `b`
# [ 58,  64]
c = tf.linalg.matvec(a, b)
b

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([ 7,  9, 11], dtype=int32)>

In [500]:
np.exp(2)

7.38905609893065